In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy import stats

import ecephys_analyses as ea 
from ecephys.signal.xarray_utils import filter_dataset_by_hypnogram

In [4]:
xr.set_options(keep_attrs=True)

In [5]:
subjects = ["Segundo", "Valentino", "Doppio", "Alessandro", "Allan"]
experiment = "sleep-homeostasis"
bsl_condition = "recovery-sleep-circadian-match"
rec_condition = "recovery-sleep"
states = ["N1", "N2"]
duration = '02:00:00'
f_range = (0.5, 4)

In [6]:
def get_subject_data(
    subject, experiment, bsl_condition, rec_condition, states, duration, f_range
):
    rec_hyp = ea.load_hypnogram(subject, experiment, rec_condition)
    rec_hyp = rec_hyp.keep_states(states).keep_first(duration)

    rec_pow = (
        ea.load_power(subject, experiment, rec_condition, "spg.nc")
        .sel(frequency=slice(*f_range))
        .sum(dim="frequency")
    )
    rec_pow = (
        filter_dataset_by_hypnogram(rec_pow, rec_hyp)
        .swap_dims({"time": "datetime"})
        .drop_vars(["time", "timedelta"])
    )

    bsl_hyp = ea.load_hypnogram(subject, experiment, bsl_condition)
    bsl_hyp = bsl_hyp.keep_states(states).keep_between(
        rec_hyp.start_time.min().strftime("%H:%M:%S"),
        rec_hyp.start_time.max().strftime("%H:%M:%S"),
    )

    bsl_pow = (
        ea.load_power(subject, experiment, bsl_condition, "spg.nc")
        .sel(frequency=slice(*f_range))
        .sum(dim="frequency")
    )
    bsl_pow = (
        filter_dataset_by_hypnogram(bsl_pow, bsl_hyp)
        .swap_dims({"time": "datetime"})
        .drop_vars(["time", "timedelta"])
    )

    diff_means = rec_pow.mean() - bsl_pow.mean()
    diff_means_std = diff_means / bsl_pow.mean()
    diff_meds = rec_pow.median() - bsl_pow.median()
    diff_meds_std = diff_meds / bsl_pow.median()

    index = pd.MultiIndex.from_product(
        [[subject], ["cortex", "hippocampus"]], names=["subject", "region"]
    )
    data = np.asarray(
        [
            [
                diff_means.mpta_wm_ref,
                diff_means_std.mpta_wm_ref,
                diff_meds.mpta_wm_ref,
                diff_meds_std.mpta_wm_ref,
            ],
            [
                diff_means.sr_wm_ref,
                diff_means_std.sr_wm_ref,
                diff_meds.sr_wm_ref,
                diff_meds_std.sr_wm_ref,
            ],
        ]
    )
    return pd.DataFrame(
        data,
        columns=["diff_means", "diff_means_std", "diff_meds", "diff_meds_std"],
        index=index,
    )

In [7]:
subj_data = list()
for subject in subjects:
    subj_data.append(get_subject_data(subject, experiment, bsl_condition, rec_condition, states, duration, f_range))
df = pd.concat(subj_data)

In [8]:
print(df.to_markdown())

|                               |   diff_means |   diff_means_std |   diff_meds |   diff_meds_std |
|:------------------------------|-------------:|-----------------:|------------:|----------------:|
| ('Segundo', 'cortex')         |     432114   |         3.00784  |    433506   |        4.21843  |
| ('Segundo', 'hippocampus')    |     171061   |         0.506753 |    183623   |        0.67479  |
| ('Valentino', 'cortex')       |      96756.3 |         1.57885  |     81744.9 |        2.10501  |
| ('Valentino', 'hippocampus')  |     163190   |         0.397626 |    176105   |        0.654904 |
| ('Doppio', 'cortex')          |     346453   |         1.77456  |    344962   |        2.6208   |
| ('Doppio', 'hippocampus')     |      99331.2 |         0.363013 |     95301.1 |        0.484065 |
| ('Alessandro', 'cortex')      |     219293   |         1.45191  |    250380   |        2.25997  |
| ('Alessandro', 'hippocampus') |     175011   |         0.450926 |    227451   |        0.834318 |


In [9]:
cx = df.xs('cortex', level=1, drop_level=False)
hc = df.xs('hippocampus', level=1, drop_level=False)

In [10]:
stats.ttest_1samp(cx, 0)

Ttest_1sampResult(statistic=array([3.91608817, 7.04797227, 3.90455726, 7.46034048]), pvalue=array([0.01730468, 0.00213671, 0.017474  , 0.00172507]))

In [11]:
stats.ttest_1samp(hc, 0)

Ttest_1sampResult(statistic=array([ 7.50046292, 12.39237772,  5.40901545,  7.61265726]), pvalue=array([0.00169048, 0.00024373, 0.00565816, 0.00159819]))